In [ ]:
# import sys
# !{sys.executable} -m pip install pip earthengine-api --no-cache-dir
# !{sys.executable} -m pip install pip geemap --no-cache-dir
# !{sys.executable} -m pip install pip rasterstats --no-cache-dir

# !{sys.executable} -m jupyter nbextension install --py --symlink --sys-prefix ipyleaflet
# !{sys.executable} -m jupyter nbextension enable --py --sys-prefix ipyleaflet
# !{sys.executable} -m jupyter nbextension list

# !{sys.executable} -m pip install pip ipyleaflet==0.13.6
# !{sys.executable} -m pip install pip -U geemap 
# !{sys.executable} -m conda install geemap -c conda-forge

In [ ]:
import ee
# ee.Authenticate()

In [ ]:
ee.Initialize()

In [ ]:
import numpy as np
import requests
import os
import pandas as pd
import rasterio
import boto3
import geopandas as gpd
import io
# from rasterstats import zonal_stats
import fiona
import rasterio.mask
import geemap.foliumap as geemap
import glob
import ipyleaflet

In [ ]:
# hide warnings
import warnings
warnings.filterwarnings('ignore')

# Read input data

In [ ]:
# define directory
out_dir = os.getcwd()
aws_s3_dir = "https://cities-cities4forests.s3.eu-west-3.amazonaws.com/data"

In [ ]:
## create map
Map = geemap.Map(height="350px")
Map.addLayerControl()
#Map.set_options('HYBRID');
# Map

In [ ]:
# read esa land cover
esa_land_cover = ee.ImageCollection('ESA/WorldCover/v100').first()
esaScale = esa_land_cover.projection().nominalScale()
builtup = esa_land_cover.eq(50)
Map.addLayer(builtup,{})


In [ ]:
# read height above nearest drainage
hand30_100 = ee.ImageCollection('users/gena/global-hand/hand-100')
hand30_1000 = ee.Image('users/gena/GlobalHAND/30m/hand-1000')
hand30_5000 = ee.Image('users/gena/GlobalHAND/30m/hand-5000')

# smoothen HAND a bit, scale varies a little in the tiles
#hand = hand30_100.mosaic().focal_mean(0.1) 
hand = hand30_1000.focal_mean(0.1) 


swbd = ee.Image('MODIS/MOD44W/MOD44W_005_2000_02_24').select('water_mask')
swbdMask = swbd.unmask().Not().focal_median(1)##.focal_max(5); ## .add(0.2);

HAND = hand.focal_max(1).focal_mode(2, 'circle', 'pixels', 5).mask(swbdMask)
Map.addLayer(HAND, {min:0,max:30}, 'HAND');

HANDbuiltup = HAND.updateMask(builtup)

thresh = 1
threshStr = str(thresh)
FlowAccu = hand # hand30_100, hand30_1000, hand30_5000
FA = 1000 # 100, 1000, 5000
FAstr = str(FA)
HANDthresh = FlowAccu.lte(thresh).focal_max(1).focal_mode(2, 'circle', 'pixels', 5).mask(swbdMask)
HANDthresh = HANDthresh.mask(HANDthresh)
HANDthreshBuiltup = HANDthresh.updateMask(builtup)

Map.addLayer(HANDthresh, {'palette':['0020ff'], 'opacity':0.8}, 'water (HAND < '+threshStr+'m, FA'+FAstr+')', False);

In [ ]:
# get list of c4f cities
boundary_georef = pd.read_csv('https://cities-cities4forests.s3.eu-west-3.amazonaws.com/data/boundaries/v_0/boundary_georef.csv')
boundary_georef

# Compute indicator

In [ ]:
this_indicator = pd.DataFrame() 

In [ ]:
# define calcuation function to get pixel counts, convert to percents and append to data frame
def CountCalcs(FC,DF):
    # reduce images to get flood and built-up pixel counts
    pixelcounts = HANDbuiltup.reduceRegions(FC,ee.Reducer.mean().setOutputs(['HANDmInBuilt_mean']),esaScale)
    pixelcounts = HANDthreshBuiltup.reduceRegions(pixelcounts,ee.Reducer.count().setOutputs(['HAND<='+threshStr+'mBuiltPixels']),esaScale)
    pixelcounts = builtup.reduceRegions(pixelcounts,ee.Reducer.count().setOutputs(['BuiltPixels']),esaScale)

    # convert pixel counts to area percentages and saves to FC as property
    def toPct(feat):
        pct = (feat.getNumber('HAND<='+threshStr+'mBuiltPixels').divide(feat.getNumber('BuiltPixels')))#.multiply(100)
        mean = feat.getNumber('HANDmInBuilt_mean')
        return feat.set({
            'PctBuiltwithin'+threshStr+'MofDrainage': pct,
            'MeanHANDinMetersofBuiltup': mean
      })

    pixelcounts = pixelcounts.map(toPct).select(['geo_id','PctBuiltwithin'+threshStr+'MofDrainage','MeanHANDinMetersofBuiltup'])

    # store in df and apend
    df = geemap.ee_to_pandas(pixelcounts)
    df = df.rename(columns={'PctBuiltwithin'+threshStr+'MofDrainage': 'GRE_4_3_percentBuiltupWithin'+threshStr+'mAboveDrainage'})
    DF = DF.append(df)
    return DF

In [ ]:
for i in range(0,len(boundary_georef)):
    print(i)
    geo_name = boundary_georef.loc[i, 'geo_name']
    print("\n geo_name: "+geo_name)
    
    boundary_id_aoi = boundary_georef.loc[i, 'geo_name']+'-'+boundary_georef.loc[i, 'aoi_boundary_name']
    boundary_id_unit = boundary_georef.loc[i, 'geo_name']+'-'+boundary_georef.loc[i, 'units_boundary_name']
        
    # process aoi level ------
    print("\n boundary_id_aoi: "+boundary_id_aoi)
    # read boundaries
    boundary_path = aws_s3_dir +'/boundaries/v_0/boundary-'+boundary_id_aoi+'.geojson'
    boundary_geo = requests.get(boundary_path).json()
    boundary_geo_ee = geemap.geojson_to_ee(boundary_geo)
    this_indicator = CountCalcs(boundary_geo_ee,this_indicator)
    
    # process unit of analysis level ------
    print("\n boundary_id_unit: "+boundary_id_unit)
    # read boundaries
    boundary_path = aws_s3_dir +'/boundaries/v_0/boundary-'+boundary_id_unit+'.geojson'
    boundary_geo = requests.get(boundary_path).json()
    boundary_geo_ee = geemap.geojson_to_ee(boundary_geo)
    this_indicator = CountCalcs(boundary_geo_ee,this_indicator)


In [ ]:
this_indicator

# Merge with indicator table

In [ ]:
# read indicator table
cities_indicators = pd.read_csv(aws_s3_dir + '/indicators/cities_indicators_v2test.csv') 
cities_indicators#.head()

In [ ]:
def merge_indicators(indicator_table, new_indicator_table, indicator_name):
    if indicator_name in indicator_table.columns:
        print("replace with new calculations")
        indicator_table.drop(indicator_name, inplace=True, axis=1)
        new_indicator_table = new_indicator_table.drop_duplicates()
        cities_indicators_df = indicator_table.merge(new_indicator_table[["geo_id",indicator_name]], 
                                                     on='geo_id', 
                                                     how='left',
                                                     validate='one_to_many')
    else:
        print("add new indicator")
        new_indicator_table = new_indicator_table.drop_duplicates()
        cities_indicators_df = indicator_table.merge(new_indicator_table[["geo_id",indicator_name]], 
                                                     on='geo_id', 
                                                     how='left',
                                                     validate='one_to_many')
    return(cities_indicators_df)

In [ ]:
cities_indicators_merged = merge_indicators(indicator_table = cities_indicators,
                                            new_indicator_table = this_indicator,
                                            indicator_name = 'GRE_4_3_percentBuiltupWithin'+threshStr+'mAboveDrainage')

In [ ]:
cities_indicators_merged

# Upload in aws s3

In [ ]:
# connect to s3
aws_credentials = pd.read_csv('/home/jovyan/PlanetaryComputerExamples/aws_credentials.csv')
aws_key = aws_credentials.iloc[0]['Access key ID']
aws_secret = aws_credentials.iloc[0]['Secret access key']

s3 = boto3.resource(
    service_name='s3',
    aws_access_key_id=aws_key,
    aws_secret_access_key=aws_secret
)

In [ ]:
# upload to aws
key_data = 'data/indicators/cities_indicators_v2test.csv'
bucket_name = 'cities-cities4forests' 
cities_indicators_merged.to_csv(
    f"s3://{bucket_name}/{key_data}",
    index=False,
    storage_options={
        "key": aws_key,
        "secret": aws_secret
    },
)

In [ ]:
# make it public
object_acl = s3.ObjectAcl(bucket_name,key_data)
response = object_acl.put(ACL='public-read')